In [ ]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
%matplotlib qt

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# define source ROI and target ROI
sROI = [[210,720],[1100,720],[720,470],[565,470]]
dROI = [[320,720],[960,720],[960,0],[320,0]]

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = np.array([[0,0]])
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = np.array([[(0,0,0),(0,0,0)]])
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

In [ ]:
# Define global status for detection
line = Line()

In [ ]:
def line_reset(_line):
    # was the line detected in the last iteration?
    _line.detected = False  
    # x values of the last n fits of the line
    _line.recent_xfitted = np.array([[0,0]])
    #average x values of the fitted line over the last n iterations
    _line.bestx = None     
    #polynomial coefficients averaged over the last n iterations
    _line.best_fit = None  
    #polynomial coefficients for the most recent fit
    _line.current_fit = np.array([[(0,0,0),(0,0,0)]])
    #radius of curvature of the line in some units
    _line.radius_of_curvature = None 
    #distance in meters of vehicle center from the line
    _line.line_base_pos = None 
    #difference in fit coefficients between last and new fits
    _line.diffs = np.array([0,0,0], dtype='float') 
    #x values for detected line pixels
    _line.allx = None  
    #y values for detected line pixels
    _line.ally = None 
    
    return _line

In [ ]:
# Undistort the image with previous calibration parameters
def Undistort(img):
    # Read in the saved objpoints and imgpoints
    dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb" ) )
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

In [ ]:
# Transform using Sobel X, S channel from HLS and V channel from HSV
def ColorSpaceTransform(img):
    # Convert to HLS color space and separate the S channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold color channel S
    thresh_min = 50
    thresh_max = 255
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel V
    v_thresh_min = 200
    v_thresh_max = 255
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= v_thresh_min) & (v_channel <= v_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, v_binary)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(v_binary == 1) | (sxbinary == 1)] = 1
    
    return combined_binary    

In [ ]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:
# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def corners_unwarp(img, sROI, dROI):
    
    # Grab the image shape
    img_size = (img.shape[1], img.shape[0])
    
    # For source points I'm grabbing the outer four detected corners
    src = np.float32([sROI[0], sROI[1], sROI[2], sROI[3]])
    # For destination points, I'm arbitrarily choosing some points to be
    # a nice fit for displaying our warped result 
    # again, not exact, but close enough for our purposes
    dst = np.float32([dROI[0], dROI[1], dROI[2], dROI[3]])
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size,flags=cv2.INTER_LINEAR)

    # Return the resulting image and matrix
    return warped, M

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2)
        #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### TO-DO: Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
      
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        #pass # Remove this when you add your function
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        err = False
    except ValueError:
        # Avoids an error if the above is not implemented fully
        err = True
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return out_img, leftx, lefty, rightx, righty, err

In [ ]:
def fit_polynomial(binary_warped):
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    
    # Find our lane pixels first
    out_img, leftx, lefty, rightx, righty, err = find_lane_pixels(binary_warped)

    if (err == False):
        ### TO-DO: Fit a second order polynomial to each using `np.polyfit` ###
        try:
            left_fit = np.polyfit(lefty, leftx, 2)
            right_fit = np.polyfit(righty, rightx, 2)
            err = False
        except TypeError:
            print('fit polynomial error!')
            left_fit = np.array([0,0,0])
            right_fit = np.array([0,0,0])
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty
            err = True

        if (err == False):
            try:
                left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
                right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
                err = False
            except TypeError:
                # Avoids an error if `left` and `right_fit` are still none or incorrect
                print('The function failed to fit a line!')
                left_fitx = 1*ploty**2 + 1*ploty
                right_fitx = 1*ploty**2 + 1*ploty
                err = True

            ## Visualization ##
            # Colors in the left and right lane regions
            out_img[lefty, leftx] = [255, 0, 0]
            out_img[righty, rightx] = [0, 0, 255]

            # Plots the left and right polynomials on the lane lines
            #plt.plot(left_fitx, ploty, color='yellow')
            #plt.plot(right_fitx, ploty, color='yellow')
    
    return out_img, left_fit, right_fit, left_fitx, right_fitx, err

In [ ]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    
    ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        err = False
    except TypeError:
        print('fit poly error!')
        left_fit = np.array([0,0,0])
        right_fit = np.array([0,0,0])
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        err = True
        
    if(err == False):    
        try:
            ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
            err = False
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty
            err = True
    
    return left_fit, right_fit, left_fitx, right_fitx, ploty, err

In [ ]:
# Use for window search of lane lines
def search_around_poly(binary_warped, left_fit, right_fit):
    result = 0
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fit, right_fit, left_fitx, right_fitx, ploty, err = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    if (err == False):
        ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        window_img = np.zeros_like(out_img)
        
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
                                  
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))
        
        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

        # Plot the polynomial lines onto the image
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')
        ## End visualization steps ##
    
    return result, left_fit, right_fit, left_fitx, right_fitx, err

In [ ]:
# calculate left and right radius, offset of vehicle to midpoint of lane (right: +ve, left: -ve)
def measure_curvature_real(img, left_fit, right_fit, left_fitx, right_fitx):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    #ploty, left_fit_cr, right_fit_cr = generate_data(ym_per_pix, xm_per_pix)
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fit_cr = left_fit
    right_fit_cr = right_fit
    
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = (0.5 / np.abs(left_fit_cr[0]))*(1+(2*left_fit_cr[0]*y_eval*ym_per_pix+left_fit_cr[1])**2)**1.5
    right_curverad = (0.5 / np.abs(right_fit_cr[0]))*(1+(2*right_fit_cr[0]*y_eval*ym_per_pix+right_fit_cr[1])**2)**1.5
    
    # Calculate offset
    offset = ((left_fitx[-1]+right_fitx[-1])/2 - img.shape[1]/2) * xm_per_pix
    
    return left_curverad, right_curverad, offset

In [ ]:
# Project back the detected lanes region to original undistorted image
def WarpProject(img, warped, _src, _dst, left_fitx, right_fitx):    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Compute the inverse perspective transform:
    src = np.float32([_src[0], _src[1], _src[2], _src[3]])
    dst = np.float32([_dst[0], _dst[1], _dst[2], _dst[3]])
    Minv = cv2.getPerspectiveTransform(src, dst)
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (warped.shape[1], warped.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    return result

In [ ]:
# overlay radius (smaller between left and right) and offset (right +ve; left -ve)
def put_Radius_Offset(img, left_curverad, right_curverad , offset):
    
    t_offset = "Vehicle Offset to Middle Lane = " + str(round(offset, 2)) + " m"
    
    # consider the smaller radius of both side
    if left_curverad < right_curverad:
        t_rad = "Radius of Lane = " + str(int(left_curverad)) + " m"
    else:
        t_rad = "Radius of Lane = " + str(int(right_curverad)) + " m"
        
    # overlay radius and offset
    cv2.putText(img, t_offset, (50, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, 255)
    cv2.putText(img, t_rad, (50, 100), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, 255)
    return img

In [ ]:
# return last n left and right polynomial coeff. and curves
def get_last_fitx_stat(n):
    global line

    _l = line.recent_xfitted.T[0][n:]
    _r = line.recent_xfitted.T[1][n:]
    left_fitx_mean = np.mean(_l)
    right_fitx_mean = np.mean(_r)
    left_fitx_std = np.std(_l)
    right_fitx_std = np.std(_r)
    
    return left_fitx_mean, left_fitx_std, right_fitx_mean, right_fitx_std

In [ ]:
def FindLaneLines(image):
    global line
       
    # apply undistortion from previous calibrated parameter of camera
    img_undist = Undistort(image)
    
    # Use Sobel X, S channel from HLS and V channel from HSV to extract lane lines
    img_space = ColorSpaceTransform(img_undist)
    
    # Select only the mask ROI
    img_space2 = np.dstack((img_space*255,img_space*255,img_space*255))
    vertices = np.array([[(sROI[0][0]-20,sROI[0][1]),(sROI[1][0]+20,sROI[1][1]),(sROI[2][0]+10, sROI[2][1]-10),(sROI[3][0]-10,sROI[3][1]-10) ]], dtype=np.int32)
    img_roi = region_of_interest(img_space2, vertices)
    
    # Perspective transform to 2D bird-eye view of lane lines
    img_unwarp, M = corners_unwarp(img_roi, sROI, dROI)
    
    # Interpolate lane lines
    # blind fit for first time
    if(len(line.recent_xfitted) == 1):
        img_fitted, left_fit, right_fit, left_fitx, right_fitx, err = fit_polynomial(img_unwarp[:,:,0])
    else:
        # smart fit with last left_fit and right_fit
        left_fit = line.current_fit[-1][0]
        right_fit = line.current_fit[-1][1]
        img_fitted, left_fit, right_fit, left_fitx, right_fitx, err = search_around_poly(img_unwarp[:,:,0], left_fit, right_fit)
        
        # go back to blind fit when smart fit failed
        if (err == True):
            img_fitted, left_fit, right_fit, left_fitx, right_fitx, err = fit_polynomial(img_unwarp[:,:,0])
     
    # only save new data to buffer if no error
    if (err == False):      
        # set window size for mean calculation
        nn = 10
        if(len(line.recent_xfitted) < nn):
            n = -len(line.recent_xfitted)
        else:
            n = -nn
        
        # calculate mean and standard deviation
        left_fitx_mean, left_fitx_std, right_fitx_mean, right_fitx_std = get_last_fitx_stat(n)
        
        #print(abs(np.mean(left_fitx) - left_fitx_mean))
        #print(left_fitx_std)
        #print(abs(np.mean(right_fitx) - right_fitx_mean))
        #print(right_fitx_std)
        #print(abs(np.mean(left_fitx) - np.mean(right_fitx)) - abs(left_fitx_mean-right_fitx_mean))
        #print(left_fitx_std + right_fitx_std)
        
        #print(np.mean(left_fitx))
        #print(left_fitx_mean)
        #print(np.mean(right_fitx))
        #print(right_fitx_mean)
        
        # save new data to buffer only if 
        # initially before window size is filled, or
        # mean (all left) is within one STDEV of historical mean of left, (same as right side), and
        # mean of width is within one STDEV of historical mean of width
        if ( n != -nn or 
            (abs(np.mean(left_fitx) - left_fitx_mean) < 50 and #left_fitx_std and
             abs(np.mean(right_fitx) - right_fitx_mean) < 50 and #right_fitx_std and
             abs(np.mean(left_fitx) - np.mean(right_fitx)) - abs(left_fitx_mean-right_fitx_mean) < 100 )): #left_fitx_std + right_fitx_std )):
            
            # store all mean of left and right x
            line.recent_xfitted = np.concatenate((line.recent_xfitted, np.array([[np.mean(left_fitx), np.mean(right_fitx)]])))

            # store all poly coeff. of left and right
            line.current_fit = np.concatenate((line.current_fit, np.array([[(left_fit[0], left_fit[1], left_fit[2]),
                                                                           (right_fit[0], right_fit[1], right_fit[2]) ]])))
            
            # store the calculated mean of poly co
            line.best_fit = [(np.mean(line.current_fit.T[0][0][n:]), np.mean(line.current_fit.T[1][0][n:]), np.mean(line.current_fit.T[2][0][n:])),
                             (np.mean(line.current_fit.T[0][1][n:]), np.mean(line.current_fit.T[1][1][n:]), np.mean(line.current_fit.T[2][1][n:]))]

    # retrieve last left_fit, right_fit, left_fitx, right_fitx    
    left_fit = line.current_fit[-1][0]
    right_fit = line.current_fit[-1][1]
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0])
    left_fitx = line.best_fit[0][0]*ploty**2 + line.best_fit[0][1]*ploty + line.best_fit[0][2]
    right_fitx = line.best_fit[1][0]*ploty**2 + line.best_fit[1][1]*ploty + line.best_fit[1][2]
    

    # Project lane lines from 2D back to undistorted image
    img_unwarp = WarpProject(image, image[:,:,0], dROI, sROI, left_fitx, right_fitx)
    
    # Measure radius and offset from detection
    left_curverad, right_curverad, offset = measure_curvature_real(image[:,:,0], left_fit, right_fit, left_fitx, right_fitx)
    
    # Overlay texts to image
    img_final = put_Radius_Offset(img_unwarp, left_curverad, right_curverad, offset)
    
    return img_final

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    #reading in an image
    
    return FindLaneLines(image)

### Test on Project Video

In [ ]:
white_output = 'test_videos_output/project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_videos/project_video.mp4").subclip(0,3)
#clip1 = VideoFileClip("test_videos/project_video.mp4")
line = line_reset(line)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Test on the "Challenge Video"

In [ ]:
white_output = 'test_videos_output/challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/challenge_video.mp4").subclip(0,3)
clip1 = VideoFileClip("test_videos/challenge_video.mp4")
line = line_reset(line)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Test on the "Harder Challenge Video"

In [ ]:
white_output = 'test_videos_output/harder_challenge_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/harder_challenge_video.mp4").subclip(0,3)
clip1 = VideoFileClip("test_videos/harder_challenge_video.mp4")
line = line_reset(line)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))